### 2a
* 3 lagen 1 input 1 hidden 1 output.
    * 3 input neurons vanwegen 3 features
    * vanwegen de duimregel mean van input en output. en de duimregel even getallen. pakken we 4 neurons in hidden layer.
    * 6 output vanwege 6 verschillende klasse
    * vanwege simpele data set zijn meer hidden layers niet nodig.

* activation
    * eindigen op softmax
    * relu voor andere lagen.
* cost function
    * categorial cross entropy
* optimizer
    * Adam
* numbers of epoch.
    * epochs zijn in het begin ingesteld op 15.
* Batch size
    * 20/80 split voor test en train
    * train
        * delen in 4. batches van 100
* Learning rate
    * 0.05


In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

df = pd.read_csv('bmi.csv', sep=',')
df = df.sample(frac=1).reset_index(drop=True)
from sklearn.model_selection import train_test_split
def convert_sex(sex):
    if(sex == 'Male'):
        return 1
    else:
        return 0
df['Gender'] = df['Gender'].apply(convert_sex)


X = df.drop(columns=['Index'])
y = df['Index']  # Target column

df.head()

,Gender,Height,Weight,Index
0,0,161,154,5
1,1,185,76,2
2,0,146,101,5
3,0,150,97,5
4,1,165,62,2


In [6]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=df['Index'], random_state=42)

print(df.dtypes)  # Controleer de types van je kolommen

y_train = to_categorical(y_train, num_classes=6)  # Assuming 6 classes
y_test = to_categorical(y_test, num_classes=6)

print(X_train.shape)
print(y_train.shape)


Gender    int64
Height    int64
Weight    int64
Index     int64
dtype: object
(400, 3)
(400, 6)


In [7]:
model = Sequential()
model.add(Dense(4, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(6, activation='softmax'))

from keras.metrics import Recall
from tensorflow.keras.metrics import AUC

model.compile(Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=[AUC()])
model.fit(X_train, y_train, batch_size=100, epochs=150, validation_split=0.1)



Epoch 1/150
4/4 [==============================] - 1s 54ms/step - loss: 24.1050 - auc_1: 0.4857 - val_loss: 19.1820 - val_auc_1: 0.4707
Epoch 2/150
4/4 [==============================] - 0s 6ms/step - loss: 16.4819 - auc_1: 0.5152 - val_loss: 11.3978 - val_auc_1: 0.5038
Epoch 3/150
4/4 [==============================] - 0s 5ms/step - loss: 9.7962 - auc_1: 0.6061 - val_loss: 6.6255 - val_auc_1: 0.7588
Epoch 4/150
4/4 [==============================] - 0s 5ms/step - loss: 7.4678 - auc_1: 0.6995 - val_loss: 5.9957 - val_auc_1: 0.7766
Epoch 5/150
4/4 [==============================] - 0s 5ms/step - loss: 6.3610 - auc_1: 0.7175 - val_loss: 4.3955 - val_auc_1: 0.8165
Epoch 6/150
4/4 [==============================] - 0s 5ms/step - loss: 4.3673 - auc_1: 0.7532 - val_loss: 3.1412 - val_auc_1: 0.8316
Epoch 7/150
4/4 [==============================] - 0s 5ms/step - loss: 3.2945 - auc_1: 0.7731 - val_loss: 2.9268 - val_auc_1: 0.7954
Epoch 8/150
4/4 [==============================] - 0s 5ms/step -

In [8]:

from sklearn.metrics import confusion_matrix
# 1. Get predictions (probabilities) from the model
y_pred = model.predict(X_test)

# 2. Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the index of the highest probability for each sample

# 3. Convert true labels (y_test) from one-hot encoded to class labels
y_test_classes = np.argmax(y_test, axis=1)

# 4. Now, compute the confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)

print(cm)


[[ 3  0  0  0  0  0]
 [ 3  0  1  0  0  0]
 [ 0  0 13  1  0  0]
 [ 0  0  7  5  2  0]
 [ 0  0  3  5 13  5]
 [ 0  0  0  0  1 38]]


### Eerste Run


model.compile(Adam(learning_rate=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=100, epochs=15, validation_split=0.1)

15 epochs 864ms

De confusion matrix zet alles in klasse 6

Conclusie: loss nog vrij hoog accuracy nog vrij laag, dus we zouden hem graag wat langer willen laten trainen, en de confusion matrix liet zien dat de accuracy hoogst was wanneer alles in klasse 6 werdt gezet. Dit betekend dat de model evaluation verkeerd niet alleen op accuracy zou moeten.

### Tweede Run

model.compile(Adam(learning_rate=0.05), loss='categorical_crossentropy', metrics=['accuracy', AUC()])
model.fit(X_train, y_train, batch_size=100, epochs=150, validation_split=0.1)

5s 929ms

De confusion matrix voorspelt in de laatste 3 klassen 1 klas omhoog en omlaag

Vanaf epoch 110 ongeveer stabiel blijft

Conclusie: Hij doet het al beter dan de eerste, dus de evaluation metrics zijn vrij goed. vanaf epoch 110 blijft hij vrij stabiel alleen springt de accuracy de hele tijd een beetje omhoog en omlaag, dus wij dachten dat de learning rate verlagen dit misschien wel op zou lossen.

Derde Run

model.compile(Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=[AUC()])
model.fit(X_train, y_train, batch_size=100, epochs=150, validation_split=0.1)

5s 902ms (vrij snel nog)

De confusion matrix ziet er niet heel slecht uit, sommige zet hij nog wel verkeerd

[[ 3  0  0  0  0  0]

 [ 0  4  0  0  0  0]

 [ 0  3  9  1  1  0]

 [ 0  0  2  6  6  0]

 [ 0  0  0  4 19  3]
 
 [ 0  0  0  0  6 33]]

met een lage learning rate flatlined de AUC rond de 120 epochs dus meer dan het huidige hebben we niet nodig.

Conclusie: Dit zijn de waardes waarmee we het beste resultaat verkrijgen. We merken alleen wel dat de dataset heel erg naar de laatste klasse leunt waardoor het NN af en toe alles in klassen 6 wilt zetten. 